In [143]:
import numpy as np
import cv2, zlib, base64, io
from PIL import Image
import json
import matplotlib.pyplot as plt
from itertools import combinations_with_replacement
import itertools  
import skimage.exposure

In [144]:
# functions for data conversion. Source: https://docs.supervise.ly/data-organization/00_ann_format_navi/04_supervisely_format_objects#bitmap

def base64_2_mask(s):
    z = zlib.decompress(base64.b64decode(s))
    n = np.fromstring(z, np.uint8)
    mask = cv2.imdecode(n, cv2.IMREAD_UNCHANGED)[:, :, 3].astype(bool)
    return mask

def mask_2_base64(mask):
    img_pil = Image.fromarray(np.array(mask, dtype=np.uint8))
    img_pil.putpalette([0,0,0,255,255,255])
    bytes_io = io.BytesIO()
    img_pil.save(bytes_io, format='PNG', transparency=0, optimize=0)
    bytes = bytes_io.getvalue()
    return base64.b64encode(zlib.compress(bytes)).decode('utf-8')

In [145]:
with open('../input/testjson/via_region_data.json') as json_file:
    labels = json.load(json_file)

In [146]:
mask_strx = labels['GABLE.213248591.jpg47177']['regions']['0']['shape_attributes']['all_points_x']
mask_stry = labels ['GABLE.213248591.jpg47177']['regions']['0']['shape_attributes']['all_points_y']

In [147]:
img = cv2.imread('../input/images/GABLE.213248591.jpg')

area = np.array([mask_strx,mask_stry])

In [148]:

points=np.stack((mask_strx,mask_stry), axis=1)

In [149]:
filled = np.zeros_like(img)
filled = cv2.fillPoly(filled, pts = [points], color =(255,255,255))


In [150]:
blur = cv2.GaussianBlur(filled, (0,0), sigmaX=2, sigmaY=2, borderType = cv2.BORDER_DEFAULT)

result = skimage.exposure.rescale_intensity(blur, in_range=(127.5,255), out_range=(0,255)).astype(np.uint8)

cv2.imwrite("filled_polygon.jpg",filled)
cv2.imwrite("filled_polygon_antialiased.jpg",result)


In [151]:
plt.figure(figsize=(20,12))
plt.imshow(img)
plt.imshow(result)